In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

import pandas as pd
from scipy.spatial.distance import cdist
import seaborn as sns
from sklearn.model_selection import cross_validate, RepeatedStratifiedKFold
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline

from BEKML import BEMKL, poly_kernel, gauss_kernel, scoring

sns.set(style='ticks', context='talk')
np.set_printoptions(precision=4, linewidth=100)

Populating the interactive namespace from numpy and matplotlib


In [4]:
data = pd.read_csv('data/wdbc.csv', names=list(range(32)), index_col=0)
display(data.head())
data.describe()

,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
0,,,,,,,,,,,,,,,,,,,,,
842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


,2,3,4,5,6,7,8,9,10,11,...,22,23,24,25,26,27,28,29,30,31
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,0.062798,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,0.007060,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,0.057700,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,0.061540,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,0.066120,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


In [13]:
X = data.iloc[:, 1:].copy()
y = data.iloc[:, 0].copy()
N, D = X.shape
y.loc[y == 'B'] = -1
y.loc[y == 'M'] = 1
y = y.astype(int)
display(X.shape, y.shape, y.mean())

(569, 30)

(569,)

-0.2548330404217926

In [14]:
kernels = [lambda A, B: gauss_kernel(A, B, 2**i) for i in range(-3, 7)]
kernels += [lambda A, B: poly_kernel(A, B, 0, i) for i in range(1, 4)]
kernels += [lambda A, B: gauss_kernel(A[:, j:j+1], B[:, j:j+1], 2**i) for i in range(-3, 7) for j in range(D)]
kernels += [lambda A, B: poly_kernel(A[:, j:j+1], B[:, j:j+1], 0, i) for i in range(1, 4) for j in range(D)]
sparse_model = BEMKL(kernels=kernels, alpha_lambda=1, beta_lambda=1, alpha_gamma=1, beta_gamma=1, alpha_omega=1e-11, beta_omega=1e+9)
normal_model = BEMKL(kernels=kernels, alpha_lambda=1, beta_lambda=1, alpha_gamma=1, beta_gamma=1, alpha_omega=1, beta_omega=1)
len(kernels)

403

In [15]:
sparse_pipeline = make_pipeline(Normalizer(), sparse_model)
normal_pipeline = make_pipeline(Normalizer(), normal_model)

In [17]:
scoring.iteration = 0
folds = RepeatedStratifiedKFold(n_splits=3, n_repeats=1)
cross_validate(sparse_pipeline, X, y, cv=folds, scoring=scoring)

0 - Non-0: 313. Total: 403. Ratio: 0.7766749379652605
1 - Non-0: 313. Total: 403. Ratio: 0.7766749379652605
2 - Non-0: 313. Total: 403. Ratio: 0.7766749379652605


{'fit_time': array([ 18.5228,  19.3738,  17.8285]),
 'score_time': array([ 1.0544,  1.0442,  1.0563]),
 'test_score': array([ 0.9053,  0.9263,  0.9206]),
 'train_score': array([ 0.9261,  0.9208,  0.9211])}

In [18]:
scoring.iteration = 0
folds = RepeatedStratifiedKFold(n_splits=3, n_repeats=1)
cross_validate(normal_pipeline, X, y, cv=folds, scoring=scoring)

0 - Non-0: 313. Total: 403. Ratio: 0.7766749379652605
1 - Non-0: 313. Total: 403. Ratio: 0.7766749379652605
2 - Non-0: 313. Total: 403. Ratio: 0.7766749379652605


{'fit_time': array([ 17.939 ,  18.2093,  17.8533]),
 'score_time': array([ 1.0448,  1.0433,  1.0431]),
 'test_score': array([ 0.9368,  0.9158,  0.9048]),
 'train_score': array([ 0.9156,  0.9235,  0.9289])}

=================

In [19]:
kernels = [lambda A, B: gauss_kernel(A, B, 2**i) for i in range(-3, 7)]
kernels += [lambda A, B: poly_kernel(A, B, 1, i) for i in range(1, 4)]
kernels += [lambda A, B: gauss_kernel(A[:, j:j+1], B[:, j:j+1], 2**i) for i in range(-3, 7) for j in range(D)]
kernels += [lambda A, B: poly_kernel(A[:, j:j+1], B[:, j:j+1], 1, i) for i in range(1, 4) for j in range(D)]
sparse_model = BEMKL(kernels=kernels, alpha_lambda=1, beta_lambda=1, alpha_gamma=1, beta_gamma=1, alpha_omega=1e-11, beta_omega=1e+9)
normal_model = BEMKL(kernels=kernels, alpha_lambda=1, beta_lambda=1, alpha_gamma=1, beta_gamma=1, alpha_omega=1, beta_omega=1)
len(kernels)

403

In [20]:
sparse_pipeline = make_pipeline(Normalizer(), sparse_model)
normal_pipeline = make_pipeline(Normalizer(), normal_model)

In [21]:
scoring.iteration = 0
folds = RepeatedStratifiedKFold(n_splits=3, n_repeats=1)
cross_validate(sparse_pipeline, X, y, cv=folds, scoring=scoring)

0 - Non-0: 403. Total: 403. Ratio: 1.0
1 - Non-0: 403. Total: 403. Ratio: 1.0
2 - Non-0: 403. Total: 403. Ratio: 1.0


{'fit_time': array([ 18.7272,  18.0432,  17.3241]),
 'score_time': array([ 1.0241,  1.0145,  0.9875]),
 'test_score': array([ 0.9211,  0.9316,  0.9153]),
 'train_score': array([ 0.9261,  0.9156,  0.9237])}

In [22]:
scoring.iteration = 0
folds = RepeatedStratifiedKFold(n_splits=3, n_repeats=1)
cross_validate(normal_pipeline, X, y, cv=folds, scoring=scoring)

0 - Non-0: 403. Total: 403. Ratio: 1.0
1 - Non-0: 403. Total: 403. Ratio: 1.0
2 - Non-0: 403. Total: 403. Ratio: 1.0


{'fit_time': array([ 17.7394,  18.0657,  18.7904]),
 'score_time': array([ 1.0112,  0.9999,  1.0231]),
 'test_score': array([ 0.9474,  0.9   ,  0.9206]),
 'train_score': array([ 0.9182,  0.934 ,  0.9289])}